In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define data generators for training and testing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split data into training and validation
)

# Set the batch size
batch_size = 16

# Create a single data generator that reads from 'train' directory and uses subfolder names as labels
train_generator = datagen.flow_from_directory(
    'skin/train',
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have multiple classes (categorical classification)
    subset='training'  # Use this for training data
)

validation_generator = datagen.flow_from_directory(
    'skin/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use this for validation data
)

# Load MobileNetV2 Lite model (remove top classification layers)
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer with dynamic class count
])

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Set the number of training epochs
epochs = 10

# Define a model checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)


Found 5962 images belonging to 6 classes.
Found 1490 images belonging to 6 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               163968    
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                    

In [5]:
# Load the best saved model (the one with the highest validation accuracy)
best_model = tf.keras.models.load_model('best_model.h5')

# Create a test data generator
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values for test data
test_generator = test_datagen.flow_from_directory(
    'skin/test',
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (categorical classification)
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = best_model.evaluate(test_generator)

# Print the test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 1644 images belonging to 6 classes.
103/103 [==============================] - 5s 38ms/step - loss: 2.3850 - accuracy: 0.4234
Test Accuracy: 42.34%


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Define a function to preprocess an image and make predictions
def predict_image(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize pixel values to [0, 1]

    # Make predictions
    predictions = model.predict(img)

    # Get the class label with the highest probability
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class[0]

# Test the model on a new image
image_path = 'download.jpeg'  # Replace with the path to your new image
predicted_class = predict_image(image_path)

# Print the predicted class
print(f"Predicted Class: {predicted_class}")


1/1 [==============================] - 1s 525ms/step
Predicted Class: 2


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define data generators for training and testing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split data into training and validation
)

# Set the batch size
batch_size = 8


# Create a single data generator that reads from 'train' directory and uses subfolder names as labels
train_generator = datagen.flow_from_directory(
    'skin/train',
    target_size=(299, 299),  # Resize images to (299, 299) for InceptionV3
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have multiple classes (categorical classification)
    subset='training'  # Use this for training data
)

validation_generator = datagen.flow_from_directory(
    'skin/train',
    target_size=(299, 299),  # Resize images to (299, 299) for InceptionV3
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use this for validation data
)

# Load InceptionV3 model with pre-trained weights (exclude top classification layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add custom classification layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer with dynamic class count
])

# Change the learning rate to a different value, e.g., 0.001
optimizer = Adam(learning_rate=0.001)

# Compile the model with the updated optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Set the number of training epochs
epochs = 15

# Define a model checkpoint to save the best model
checkpoint = ModelCheckpoint('best_inceptionv3_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)


Found 5962 images belonging to 6 classes.
Found 1490 images belonging to 6 classes.
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_7   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_7 (Dropout)         (None, 2048)              0         
                                                                 
 dense_14 (Dense)            (None, 128)               262272    
                                                                 
 dense_15 (Dense)            (None, 6)                 774       
                                                                 
Total params: 22,065,830
Trainable p

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = tf.keras.models.load_model('best_inceptionv3_model.h5')

# Define data generator for testing
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values for test data
test_generator = test_datagen.flow_from_directory(
    'skin/test',
    target_size=(299, 299),  # Resize images to (299, 299) for InceptionV3
    batch_size=16,  # You can adjust the batch size as needed
    class_mode='categorical',  # Assuming you have multiple classes (categorical classification)
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

# Print the test accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 1644 images belonging to 6 classes.
103/103 [==============================] - 11s 94ms/step - loss: 1.2866 - accuracy: 0.4398
Test Accuracy: 43.98%


In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_inceptionv3_model.h5')

# Define a function to preprocess an image and make predictions
def predict_image(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(299, 299))  # Resize to (299, 299) for InceptionV3
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize pixel values to [0, 1]

    # Make predictions
    predictions = model.predict(img)

    # Get the class label with the highest probability
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class[0]

# Specify the path to the individual image you want to classify
image_path = r'D:\skindisease\skin\test\Normal\0_0_aidai_0084.jpg'  # Replace with the path to your single image
predicted_class = predict_image(image_path)

# Print the predicted class
print(f"Predicted Class: {predicted_class}")


1/1 [==============================] - 1s 901ms/step
Predicted Class: 5
